In [1]:
import pandas as pd
import numpy as np
from pypfopt.expected_returns import returns_from_prices
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

### Creating portfolio based on LSTM prediction

In [3]:
data = pd.read_excel('Stocks_Adj_Close_Data.xlsx', sheet_name = 'LSTM')
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y') 
data.set_index('Date', inplace=True)
data = data.fillna(method='ffill')  
data = data.fillna(method='bfill')

In [4]:
actual_returns = data[['sp', 'ftse', 'nikkei', 'apple', 'goog', 'amzn', 'amd']].pct_change().dropna()
predicted_returns = data[['pred_sp', 'pred_ftse', 'pred_nikkei', 'pred_apple', 'pred_goog', 'pred_amzn', 'pred_amd']].pct_change().dropna()

mean_actual_returns = actual_returns.mean()
mean_predicted_returns = predicted_returns.mean()

In [6]:
short_candidates = predicted_returns.mean().nsmallest(2) 

print("Short Sell Candidates based on Predictions:")
print(short_candidates)

Short Sell Candidates based on Predictions:
pred_nikkei   -0.002125
pred_sp       -0.001546
dtype: float64


We decided to sell the Nikkei and Sp at the higher price and buy at the lower price at the end

In [27]:
initial_nikkei_price = data['pred_nikkei'].iloc[0]
final_nikkei_price = data['pred_nikkei'].iloc[-1]
initial_sp_price = data['pred_sp'].iloc[0]
final_sp_price = data['pred_sp'].iloc[-1]

profit_nikkei = initial_nikkei_price - final_nikkei_price
profit_sp = initial_sp_price - final_sp_price

total_profit = profit_nikkei + profit_sp

print(f"Profit from Nikkei: ${profit_nikkei:.2f}")
print(f"Profit from S&P 500: ${profit_sp:.2f}")
print(f"Total Profit from buying low and selling high: ${total_profit:.2f}")

Profit from Nikkei: $1793.66
Profit from S&P 500: $170.10
Total Profit from buying low and selling high: $1963.76


In [50]:
import numpy as np

predicted_returns = {
    'pred_ftse': 0.000092,
    'pred_apple': 0.002119,
    'pred_goog': 0.001572,
    'pred_amzn': 0.000545,
    'pred_amd': 0.003002,  # High return but considered risky
    'pred_sp': -0.001546,
    'pred_nikkei': -0.002125
}

predicted_prices = {
    'pred_ftse': 7700.41,
    'pred_apple': 179.52,
    'pred_goog': 144.29,
    'pred_amzn': 177.55,
    'pred_amd': 192.17,
    'pred_sp': 5082.58,
    'pred_nikkei': 39235.92
}

closing_prices = {
    'pred_ftse': 7715.91,
    'pred_apple': 188.07,
    'pred_goog': 150.00,
    'pred_amzn': 149.36,
    'pred_amd': 205.27,
    'pred_sp': 4912.48,
    'pred_nikkei': 37442.26
}

total_investment = 150000

nikkei_share_cost = predicted_prices['pred_nikkei']
remaining_funds = total_investment - nikkei_share_cost

categories = {'negative': [], 'low_to_moderate': [], 'high': [], 'volatile': []}
for stock, return_ in predicted_returns.items():
    if stock == 'pred_nikkei':  
        continue
    if return_ < 0:
        categories['negative'].append(stock)
    elif return_ < 0.0002:
        categories['low_to_moderate'].append(stock)
    else:
        categories['high'].append(stock)
categories['volatile'].append('pred_amd')  

budget_allocation = {
    'negative': remaining_funds * 0.3,
    'low_to_moderate': remaining_funds * 0.4,
    'high': remaining_funds * 0.2,  
    'volatile': remaining_funds * 0.1 
}


stock_investment = {'pred_nikkei': nikkei_share_cost}  
for category, stocks in categories.items():
    amount_per_stock = budget_allocation[category] / len(stocks)
    for stock in stocks:
        price = predicted_prices[stock]
        shares = np.floor(amount_per_stock / price)
        stock_investment[stock] = shares * price


total_allocated = sum(stock_investment.values())
print(f"Total Allocated: ${total_allocated:.2f}")


print("Investment per Stock (Adjusted for Whole Shares):")
for stock, investment in stock_investment.items():
    print(f"{stock}: ${investment:.2f}, Shares: {investment / predicted_prices[stock]}")

Total Allocated: $135559.81
Investment per Stock (Adjusted for Whole Shares):
pred_nikkei: $39235.92, Shares: 1.0
pred_sp: $30495.48, Shares: 6.0
pred_ftse: $38502.05, Shares: 5.000000000000001
pred_apple: $5385.60, Shares: 30.0
pred_goog: $5483.02, Shares: 38.0
pred_amzn: $5504.05, Shares: 31.0
pred_amd: $10953.69, Shares: 57.0


In [34]:
import numpy as np


predicted_prices = {
    'pred_ftse': 7700.41,
    'pred_apple': 179.52,
    'pred_goog': 144.29,
    'pred_amzn': 177.55,
    'pred_amd': 192.17,
    'pred_sp': 5082.58,
    'pred_nikkei': 39235.92
}


stock_investment = {
    'pred_nikkei': 39235.92,  # Shares: 1
    'pred_sp': 30495.48,      # Shares: 6
    'pred_ftse': 38502.05,    # Shares: 5
    'pred_apple': 5385.60,    # Shares: 30
    'pred_goog': 5483.02,     # Shares: 38
    'pred_amzn': 5504.05,     # Shares: 31
    'pred_amd': 10953.69      # Shares: 57
}


total_allocated = sum(stock_investment.values())
remaining_funds = total_investment - total_allocated


middle_return_stocks = ['pred_ftse', 'pred_apple', 'pred_goog', 'pred_amzn']  


for stock in middle_return_stocks:
    if remaining_funds > 0:
        price = predicted_prices[stock]
        additional_shares = np.floor(remaining_funds / price)
        additional_investment = additional_shares * price
        stock_investment[stock] += additional_investment
        remaining_funds -= additional_investment


total_allocated = sum(stock_investment.values())
remaining_funds = total_investment - total_allocated

print(f"Total Allocated after redistribution: ${total_allocated:.2f}")
print("Investment per Stock (Adjusted for Whole Shares, after redistribution):")
for stock, investment in stock_investment.items():
    print(f"{stock}: ${investment:.2f}, Shares: {investment / predicted_prices[stock]}")

Total Allocated after redistribution: $149902.46
Investment per Stock (Adjusted for Whole Shares, after redistribution):
pred_nikkei: $39235.92, Shares: 1.0
pred_sp: $30495.48, Shares: 6.0
pred_ftse: $46202.46, Shares: 6.000000000000001
pred_apple: $12027.84, Shares: 67.0
pred_goog: $5483.02, Shares: 38.00000000000001
pred_amzn: $5504.05, Shares: 31.0
pred_amd: $10953.69, Shares: 57.00000000000001


In [44]:
import numpy as np


predicted_prices = {
    'pred_ftse': 7700.41, 'pred_apple': 179.52, 'pred_goog': 144.29,
    'pred_amzn': 177.55, 'pred_amd': 192.17, 'pred_sp': 5082.58, 'pred_nikkei': 39235.92
}

closing_prices = {
    'pred_ftse': 7715.91, 'pred_apple': 188.07, 'pred_goog': 150.00,
    'pred_amzn': 149.36, 'pred_amd': 205.27, 'pred_sp': 4912.48, 'pred_nikkei': 37442.26
}


stock_investment = {
    'pred_ftse': 38502.05,    # Shares: 5
    'pred_apple': 5385.60,    # Shares: 30
    'pred_goog': 5483.02,     # Shares: 38
    'pred_amzn': 5504.05,     # Shares: 31
    'pred_amd': 10953.69      # Shares: 57
}

end_of_month_values = {stock: (stock_investment[stock] / predicted_prices[stock]) * closing_prices[stock] for stock in stock_investment}
total_end_of_month_value = sum(end_of_month_values.values())

total_initial_investment = sum(stock_investment.values())


initial_nikkei_price = predicted_prices['pred_nikkei']
final_nikkei_price = closing_prices['pred_nikkei']
initial_sp_price = predicted_prices['pred_sp']
final_sp_price = closing_prices['pred_sp']

profit_nikkei = initial_nikkei_price - final_nikkei_price
profit_sp = initial_sp_price - final_sp_price

total_swing_profit = profit_nikkei + profit_sp


total_end_of_month_value += total_swing_profit


total_return = total_end_of_month_value - total_initial_investment
return_percentage = (total_return / total_initial_investment) * 100
total_return_of_portfel = total_swing_profit + total_return

print(f"End-of-Month Portfolio Value (excluding Nikkei and S&P): ${total_end_of_month_value:.2f}")
print(f"Total Initial Investment (excluding Nikkei and S&P): ${total_initial_investment:.2f}")
print(f"Profit from Swing Trading Nikkei and S&P: ${total_swing_profit:.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Return Percentage: {return_percentage:.2f}%")
print(f"Total Return (including Swing Trading): ${total_return_of_portfel}")

End-of-Month Portfolio Value (excluding Nikkei and S&P): $68215.96
Total Initial Investment (excluding Nikkei and S&P): $65828.41
Profit from Swing Trading Nikkei and S&P: $1963.76
Total Return: $2387.55
Return Percentage: 3.63%
Total Return (including Swing Trading): $4351.3099999999995


### Actual prices

In [42]:

initial_nikkei_price = data['nikkei'].iloc[0]
final_nikkei_price = data['nikkei'].iloc[-1]
initial_sp_price = data['sp'].iloc[0]
final_sp_price = data['sp'].iloc[-1]


profit_nikkei = initial_nikkei_price - final_nikkei_price
profit_sp = initial_sp_price - final_sp_price


total_profit = profit_nikkei + profit_sp


print(f"Profit from Nikkei: ${profit_nikkei:.2f}")
print(f"Profit from S&P 500: ${profit_sp:.2f}")
print(f"Total Profit from buying low and selling high: ${total_profit:.2f}")

Profit from Nikkei: $1762.41
Profit from S&P 500: $218.66
Total Profit from buying low and selling high: $1981.07


In [48]:
import numpy as np


actual_prices = {
    'ftse':  7952.60 , 'apple':  171.48 , 'goog':  152.26 ,
    'amzn':  180.38 , 'amd':  180.49 , 'sp':  5254.35 , 'nikkei':  40168.07 
}

closing_prices = {
    'ftse':  8144.13 , 'apple':  170.33 , 'goog':  164.64 ,
    'amzn':  175.00 , 'amd':  158.38 , 'sp':  5035.69 , 'nikkei':  38405.66 
}

stock_investment = {
    'ftse': 39763,    # Shares: 5
    'apple': 5144.4,    # Shares: 30
    'goog': 5785.88,     # Shares: 38
    'amzn': 5591.78,     # Shares: 31
    'amd': 10287.93     # Shares: 57
}


end_of_month_values = {stock: (stock_investment[stock] / actual_prices[stock]) * closing_prices[stock] for stock in stock_investment}
total_end_of_month_value = sum(end_of_month_values.values())


total_initial_investment = sum(stock_investment.values())


initial_nikkei_price = actual_prices['nikkei']
final_nikkei_price = closing_prices['nikkei']
initial_sp_price = actual_prices['sp']
final_sp_price = closing_prices['sp']

profit_nikkei = initial_nikkei_price - final_nikkei_price
profit_sp = initial_sp_price - final_sp_price

total_swing_profit = profit_nikkei + profit_sp


total_end_of_month_value += total_swing_profit


total_return = total_end_of_month_value - total_initial_investment
return_percentage = (total_return / total_initial_investment) * 100
total_return_of_portfel = total_swing_profit + total_return


print(f"End-of-Month Portfolio Value (excluding Nikkei and S&P): ${total_end_of_month_value:.2f}")
print(f"Total Initial Investment (excluding Nikkei and S&P): ${total_initial_investment:.2f}")
print(f"Profit from Swing Trading Nikkei and S&P: ${total_swing_profit:.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Return Percentage: {return_percentage:.2f}%")
print(f"Total Return (including Swing Trading): ${total_return_of_portfel}")

End-of-Month Portfolio Value (excluding Nikkei and S&P): $68520.60
Total Initial Investment (excluding Nikkei and S&P): $66572.99
Profit from Swing Trading Nikkei and S&P: $1981.07
Total Return: $1947.61
Return Percentage: 2.93%
Total Return (including Swing Trading): $3928.6799999999976
